In [ ]:
# def compose(*funcs):
#     if funcs:
#         return reduce(lambda f,g:lambda *a,**kw:g(f(*a,**kw)),funcs)
#     else:
#         raise ValueError('No function to compose.')

In [ ]:
# class A(object):
#     def __init__(self):
#         print("A")
# class B(object):
#     def __init__(self):
#         print("B")
# class C(object):
#     def __init__(self):
#         super(C,self).__init__()

In [ ]:
import cv2
import tensorflow as tf
import random
import numpy as np
import time
import os
import tensorflow.contrib.slim as slim
import DataGenerator


In [ ]:
BATCH_SIZE=10
BN_SIZE=4
GROWTH_RATE=32
KEEP_PROB=1.0
START_CHANNEL=64
IS_TRAIN=True
CROP_SIZE=160
NUM_FRAMES_PER_CLIP=16
NUM_CLASSES=8

In [ ]:
def convS(_X,out_channels,kernel_size=[1,3,3],stride=1,padding='VALID'):
    
#     weights_S= slim.model_variable('weights', shape=[1,kernel_size[0],kernel_size[1],kernel_size[2],1], 
#                         initializer=tf.contrib.layers.xavier_initializer(), 
#                         regularizer=slim.l2_regularizer(0.0005),
#                         device='/CPU:0')
#     return tf.nn.conv3d(l_input, weights_S, strides=[1, stride, stride, stride, 1], padding=padding)
    
    return slim.conv3d(_X,out_channels,kernel_size=kernel_size,stride=stride,padding=padding,
                                     weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                   biases_initializer=None)

def convT(_X,out_channels,kernel_size=[3,1,1],stride=1,padding='VALID'):
    return slim.conv3d(_X,out_channels,kernel_size=kernel_size,stride=stride,padding=padding,
                         weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                       biases_initializer=None)


In [ ]:
def compute_accuracy(logit,labels):
    correct=tf.equal(tf.argmax(logit,1),labels)
    acc=tf.reduce_mean(tf.cast(correct,tf.float32))
    return acc
def compute_loss(logit,label):
    cross_entropy_mean=tf.reduce_mean(
                    tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label,logits=logit))
    weight_loss=tf.losses.get_regularization_loss()
    total_loss=cross_entropy_mean+weight_loss
    return total_loss

In [ ]:
# def ST_A(_X,out_channels,s_kernel,t_kernel,stride,padding):
#     x=convS(_X,out_channels,s_kernel,stride,padding)
#     x=tf.layers.batch_normalization(x,training=IS_TRAIN)
#     x=tf.nn.relu(x)
#     x=convT(x,out_channels,t_kernel,stride,padding)
#     x=tf.layers.batch_normalization(x,training=IS_TRAIN)
#     x=tf.nn.relu(x)
#     return x
    
def ST_B(_X,out_channels,s_kernel,t_kernel,stride,padding):
    
   
    tmp_x=convS(_X,out_channels,s_kernel,stride,padding)
    tmp_x=tf.layers.batch_normalization(tmp_x,training=IS_TRAIN)
    tmp_x=tf.nn.relu(tmp_x)
    
    x=convT(_X,out_channels,t_kernel,stride,padding)
    x=tf.layers.batch_normalization(x,training=IS_TRAIN)
    x=tf.nn.relu(x)
    
    return x+tmp_x
    
def ST_C(_X,out_channels,s_kernel,t_kernel,stride,padding):
    
    x=convS(_X,out_channels,s_kernel,stride,padding)
    x=tf.layers.batch_normalization(x,training=IS_TRAIN)
    x=tf.nn.relu(x)
    
   
    tmp_x=convT(x,out_channels,t_kernel,stride,padding)
    tmp_x=tf.layers.batch_normalization(tmp_x,training=IS_TRAIN)
    tmp_x=tf.nn.relu(tmp_x)
    
    return x+tmp_x
def p3d(_X,out_channels,kernel_size=[3,3,3],stride=1,padding='SAME',tp='B'):
    s_kernel=kernel_size[1:3]
    s_kernel.insert(0,1)
    t_kernel=[1,1]
    t_kernel.insert(0,kernel_size[0])
    if tp=='B':
        return ST_B(_X,out_channels,s_kernel,t_kernel,stride,padding)
    else:
        return ST_C(_X,out_channels,s_kernel,t_kernel,stride,padding)

In [ ]:
def make_block_layer(_X,nums_inchannel,keep_prob=1,tp_idx=0):
    #Caution: Do Not use tf.contrib.layers.batch_norm() !!!
    out=tf.layers.batch_normalization(_X,training=IS_TRAIN)
    out=tf.nn.relu(out)
    out=slim.conv3d(out,BN_SIZE*GROWTH_RATE,[1,1,1],stride=1,biases_initializer=None)
    #Here,we use 'pseudo-3d convolution' instead of traditional 3D convolution(eg. slim.conv3d)
    if tp_idx % 2==0:
        out=p3d(out,GROWTH_RATE,padding='SAME',tp='B')
    else:
        out=p3d(out,GROWTH_RATE,padding='SAME',tp='C')
        #out=slim.conv3d(out,GROWTH_RATE,[3,3,3],stride=1,padding='SAME',biases_initializer=None)
    if(keep_prob!=1):
        out=slim.dropout(out,keep_prob=keep_prob)
    return tf.concat([_X,out],axis=-1)

In [ ]:
def build_block(_X,block_num,nums_inchannel):
    tp_idx=0
    for i in range(block_num):
        _X=make_block_layer(_X,nums_inchannel=nums_inchannel,keep_prob=KEEP_PROB,tp_idx=tp_idx)
        tp_idx+=1
    return _X

In [ ]:
def TTL(_X,depth=(1,3,4)):
    #As mentioned above,I replace traditional 3D conv by 'pseudo-3d conv' to achive parameters efficiency.
    y1=tf.layers.batch_normalization(_X,training=IS_TRAIN)
    y1=tf.nn.relu(y1)
    y1=slim.conv3d(y1,128,[depth[0],1,1],stride=1,biases_initializer=None) 
    
    
#     y2=tf.layers.batch_normalization(_X,training=IS_TRAIN)
#     y2=tf.nn.relu(y2)
    y2=p3d(_X,128,[depth[1],3,3],stride=1,tp='B')
    #y2=slim.conv3d(y2,128,[depth[1],3,3],stride=1,biases_initializer=None)
    
#     y3=tf.layers.batch_normalization(_X,training=IS_TRAIN)
#     y3=tf.nn.relu(y3)
    y3=p3d(_X,128,[depth[2],3,3],stride=1,tp='C')
    #y3=slim.conv3d(y3,128,[depth[2],3,3],stride=1,biases_initializer=None)
    
#     y4=tf.layers.batch_normalization(_X,training=IS_TRAIN)
#     y4=tf.nn.relu(y4)
    
    return tf.concat([y1,y2,y3],axis=-1)

In [ ]:
def Transition(_X,in_channels):
    _X=tf.layers.batch_normalization(_X,training=IS_TRAIN)
    _X=tf.nn.relu(_X)
    _X=slim.conv3d(_X,in_channels,kernel_size=[1,1,1],stride=1,biases_initializer=None)
    _X=slim.avg_pool3d(_X,[2,2,2],stride=2,padding='SAME')
    return _X

In [ ]:
def count2():
    print np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])

In [ ]:
def inference_t3d(_X,block_config=(6, 12, 24, 16)):
    
    with slim.arg_scope([slim.conv3d],
                       weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                        weights_regularizer=slim.l2_regularizer(0.0005)): 
        out=slim.conv3d(_X,START_CHANNEL,[3,7,7],stride=[1,2,2],padding='SAME',biases_initializer=None)
       
        out=tf.layers.batch_normalization(out,training=IS_TRAIN)
        out=tf.nn.relu(out)
        out=slim.max_pool3d(out,kernel_size=[3,3,3],stride=2,padding='SAME')
        in_channels=START_CHANNEL
        
        for i, num_layers in enumerate(block_config):
            
            out=build_block(out,num_layers,in_channels)
            
            in_channels=in_channels+GROWTH_RATE*num_layers
            if i!=len(block_config)-1:
                if i==0:
                    out=TTL(out,(1,3,6))
                else:
                    out=TTL(out)
                
                in_channels=128*3
                out=Transition(out,in_channels // 2)
                in_channels=in_channels // 2
                
                
        
        out=tf.layers.batch_normalization(out,training=IS_TRAIN)
        out=tf.nn.relu(out)
        #Standard input shape=[BATCH,NUM_CLIP=16,HEIGHT=160,WIDTH=160,RGB=3],makes that kernel_size of AVG_POOL equals '5'
        #If you are about to change size of input,changing the kernel size of 'avg_pool3d' simultaneously.
        out=slim.avg_pool3d(out,kernel_size=[1,5,5])
        out=tf.reshape(out,[out.get_shape().as_list()[0],-1])
        
        out=slim.fully_connected(out,NUM_CLASSES)
        return out

In [ ]:
#define dataloader to fetch data for each batch 
dataloader=DataGenerator.DataGenerator(filename='train.list',
                                batch_size=BATCH_SIZE,
                                num_frames_per_clip=NUM_FRAMES_PER_CLIP,
                                shuffle=True,is_da=False)
# #function to generate one-hot label
# def convertonehot(label):
#     return (np.arange(NUM_CLASSES)==label[:,None]).astype(np.integer)

In [ ]:
print(BATCH_SIZE)
print(IS_TRAIN)

In [ ]:
#Cell for Training!
USE_PRETRAIN=False
MODEL_PATH='./'
MAX_STEPS=5000

with tf.Graph().as_default():
    global_step=tf.get_variable('global_step',[],initializer=tf.constant_initializer(0),trainable=False)
    input_placeholder=tf.placeholder(tf.float32,shape=[BATCH_SIZE,NUM_FRAMES_PER_CLIP,CROP_SIZE,CROP_SIZE,3])
    label_placeholder=tf.placeholder(tf.int64,shape=[BATCH_SIZE])

    logit=inference_t3d(input_placeholder)
    
    #define loss:
    loss=compute_loss(logit, label_placeholder)
    tf.summary.scalar('loss',loss)
#     tf.summary.scalar('loss_total',total_loss)
    acc=compute_accuracy(logit,label_placeholder)
    tf.summary.scalar('accuracy',acc)
    
    #define lr dacay and optimizer:
    learning_rate = tf.train.exponential_decay(0.0002,
                                               global_step,decay_steps=450,decay_rate=0.3,staircase=True)
    opt_stable=tf.train.AdamOptimizer(learning_rate)
    
    #build dependecy which updating paras before training:
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optim_op=opt_stable.minimize(loss,global_step=global_step,var_list=tf.trainable_variables())
    
    saver=tf.train.Saver(tf.global_variables())
    init=tf.global_variables_initializer()
    config=tf.ConfigProto()
    gpu_options = tf.GPUOptions(allow_growth=True)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))  
    sess.run(init)
   
    if USE_PRETRAIN:
        saver.restore(sess,MODEL_PATH)
        print('Checkpoint reloaded.')
    else:
        print('Train from scratch.')
    merged=tf.summary.merge_all()
    #Using tensorboard to trace your training path.
    train_writer=tf.summary.FileWriter('./visual_t3d/train',sess.graph)
#     test_writer=tf.summary.FileWriter('./visual_logs/test',sess.graph)
    
    duration=0
    print('Start training.')
    for step in xrange(1,MAX_STEPS):
        
        start_time=time.time()
        train_images,train_labels,_=dataloader.next_batch()
        sess.run(optim_op,feed_dict={
                        input_placeholder:train_images,
                        label_placeholder:train_labels})
        duration+=time.time()-start_time
        
        if step!=0 and step % 10==0:
            curacc,curloss=sess.run([acc,loss],feed_dict={
                        input_placeholder:train_images,
                        label_placeholder:train_labels})
            print('Step %d: %.2f sec -->loss : %.4f =====acc : %.2f' % (step, duration,np.mean(curloss),curacc))
            duration=0
            
        if step!=0 and step % 50==0:
            mer=sess.run(merged,feed_dict={
                        input_placeholder:train_images,
                        label_placeholder:train_labels})
            train_writer.add_summary(mer, step)
            
        if step >1000 and step % 500==0 or (step+1)==MAX_STEPS:
            saver.save(sess,'./T3DBN_TFCKPT_ITER_{}'.format(step),global_step=step)
        
    print('done')   
            

In [ ]:
###############################################
###############################################

In [ ]:
#Cell for Testing!
BATCH_SIZE=10
#Essential:set 'IS_TRAIN' -> False
IS_TRAIN=False
testloader=DataGenerator.DataGenerator(filename='test.list',
                                batch_size=BATCH_SIZE,
                                num_frames_per_clip=NUM_FRAMES_PER_CLIP,
                                shuffle=False,is_da=False)
tf.reset_default_graph()
print('IS_TRAIN:',IS_TRAIN)
print('KEEP_PROB:',KEEP_PROB)
USE_PRETRAIN=True
# MODEL_PATH='./GT_TFCKPT_ITER_3000-3000'
with tf.Graph().as_default():
    global_step=tf.get_variable('global_step',[],initializer=tf.constant_initializer(0),trainable=False)
    input_placeholder=tf.placeholder(tf.float32,shape=[BATCH_SIZE,NUM_FRAMES_PER_CLIP,CROP_SIZE,CROP_SIZE,3])
    label_placeholder=tf.placeholder(tf.int64,shape=[BATCH_SIZE])

    logit=inference_t3d(input_placeholder)
    
    acc=compute_accuracy(logit,label_placeholder)
    init=tf.global_variables_initializer()
    saver=tf.train.Saver(tf.global_variables())
    
    config=tf.ConfigProto()
    gpu_options = tf.GPUOptions(allow_growth=True)
    sess = tf.Session()  
    sess.run(init)
   
    saver.restore(sess,'./T3DBN_TFCKPT_ITER_3000-3000')
    print('Start Testing.')
    
    print('l={}'.format(testloader.len/BATCH_SIZE))
    resacc=0
    c=0
    for step in xrange(testloader.len/BATCH_SIZE):
        
        
        test_images,test_labels,_=testloader.next_batch()
        
        curacc=sess.run(acc,feed_dict={
                        input_placeholder:test_images,
                        label_placeholder:test_labels})
        print(curacc)
        resacc+=curacc
        c+=1
        
    print('Accuracy:{:.2f}'.format(resacc/c))   
            